In [1]:
# import necessary libraries
import numpy as np
import os
import tensorflow as tf
from tensorflow import keras
from keras.layers import Conv2D,Dense,Input,Flatten,Reshape
from keras.models import Model

In [2]:
# Load datasets and reshape it
(x_train,y_train),(x_test,y_test) = keras.datasets.mnist.load_data()

x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

11490434/11490434 [==============================] - 0s 0us/step


In [3]:
# Build Autoencoder Model

input_shape = (28,28,1)
latent_dim = 32

# Encoder
inputs = Input(shape=input_shape)
x = Conv2D(32,(3,3),activation='relu',padding='same')(inputs)
x = Flatten()(x)
encoded = Dense(latent_dim)(x)

# Decoder
x = Dense(28* 28* 32)(encoded)
x = Reshape((28,28,32))(x)
x = Conv2D(32,(3,3),activation='relu',padding='same')(x)
decoded = Conv2D(1,(3,3),activation='sigmoid',padding='same')(x)

autoencoder = Model(inputs,decoded)
autoencoder.compile(optimizer='adam',loss='mean_squared_error',metrics=['accuracy'])

In [4]:
# Train autoencoder model
autoencoder.fit(x_train.reshape(-1,28,28,1),y_train,epochs=10,batch_size=128,shuffle=True,validation_data=(x_test.reshape(-1,28,28,1),y_test))

Epoch 1/10
469/469 [==============================] - 17s 15ms/step - loss: 20.3125 - accuracy: 0.1123 - val_loss: 20.2422 - val_accuracy: 0.1135
Epoch 2/10
469/469 [==============================] - 7s 14ms/step - loss: 20.2774 - accuracy: 0.1124 - val_loss: 20.2422 - val_accuracy: 0.1135
Epoch 3/10
469/469 [==============================] - 6s 14ms/step - loss: 20.2774 - accuracy: 0.1124 - val_loss: 20.2422 - val_accuracy: 0.1135
Epoch 4/10
469/469 [==============================] - 7s 15ms/step - loss: 20.2774 - accuracy: 0.1124 - val_loss: 20.2422 - val_accuracy: 0.1135
Epoch 5/10
469/469 [==============================] - 7s 14ms/step - loss: 20.2774 - accuracy: 0.1124 - val_loss: 20.2422 - val_accuracy: 0.1135
Epoch 6/10
469/469 [==============================] - 7s 15ms/step - loss: 20.2774 - accuracy: 0.1124 - val_loss: 20.2422 - val_accuracy: 0.1135
Epoch 7/10
469/469 [==============================] - 7s 15ms/step - loss: 20.2774 - accuracy: 0.1124 - val_loss: 20.2422 - val_a

In [5]:
# Now Extract features and build classification head
encoder = Model(inputs,encoded)
encoder.compile(optimizer='adam',loss='mean_squared_error',metrics=['accuracy'])
encoder.fit(x_train.reshape(-1,28,28,1),y_train,epochs=5,batch_size=128,shuffle=True,validation_data=(x_test.reshape(-1,28,28,1),y_test))

Epoch 1/5
469/469 [==============================] - 3s 5ms/step - loss: 7.8463 - accuracy: 0.0437 - val_loss: 2.4262 - val_accuracy: 0.0582
Epoch 2/5
469/469 [==============================] - 2s 5ms/step - loss: 2.1059 - accuracy: 0.0415 - val_loss: 1.8004 - val_accuracy: 0.0284
Epoch 3/5
469/469 [==============================] - 2s 5ms/step - loss: 1.7331 - accuracy: 0.0436 - val_loss: 1.6174 - val_accuracy: 0.0493
Epoch 4/5
469/469 [==============================] - 2s 5ms/step - loss: 1.5936 - accuracy: 0.0456 - val_loss: 1.5300 - val_accuracy: 0.0337
Epoch 5/5
469/469 [==============================] - 3s 6ms/step - loss: 1.5074 - accuracy: 0.0450 - val_loss: 1.4749 - val_accuracy: 0.0431


This loss won't be optimised

In [6]:
# Classiication head on the top of encoder
classification_inputs = Input(shape=(latent_dim,))
x = Dense(64,activation='relu')(classification_inputs)
outputs = Dense(10,activation='softmax')(x)

classifier = Model(classification_inputs,outputs)
classifier.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [7]:
# Train classification model
encoded_train_features = encoder.predict(x_train.reshape(-1,28,28,1))
encoded_test_features = encoder.predict(x_test.reshape(-1,28,28,1))

classifier.fit(encoded_train_features,y_train,epochs=100,shuffle=True,batch_size=128,validation_data=(encoded_test_features,y_test))

313/313 [==============================] - 1s 2ms/step
Epoch 1/100
469/469 [==============================] - 2s 3ms/step - loss: 2.0945 - accuracy: 0.2053 - val_loss: 1.8495 - val_accuracy: 0.2996
Epoch 2/100
469/469 [==============================] - 1s 3ms/step - loss: 1.6655 - accuracy: 0.3508 - val_loss: 1.5476 - val_accuracy: 0.3568
Epoch 3/100
469/469 [==============================] - 2s 4ms/step - loss: 1.4849 - accuracy: 0.4092 - val_loss: 1.4608 - val_accuracy: 0.4176
Epoch 4/100
469/469 [==============================] - 2s 4ms/step - loss: 1.4282 - accuracy: 0.4325 - val_loss: 1.4139 - val_accuracy: 0.4372
Epoch 5/100
469/469 [==============================] - 2s 3ms/step - loss: 1.4085 - accuracy: 0.4400 - val_loss: 1.4125 - val_accuracy: 0.4495
Epoch 6/100
469/469 [==============================] - 1s 3ms/step - loss: 1.3962 - accuracy: 0.4496 - val_loss: 1.4127 - val_accuracy: 0.4433
Epoch 7/100
469/469 [==============================] - 2s 3ms/step - loss: 1.3937 - acc

In [8]:
# Evaluate model
test_loss,test_accuracy = classifier.evaluate(encoded_test_features,y_test)
print(f"Test accuracy : {test_accuracy}")

313/313 [==============================] - 2s 5ms/step - loss: 1.3265 - accuracy: 0.4896
Test accuracy : 0.4896000027656555
